In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2024-10-30 13:28:56--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.24.59, 2404:6800:4006:804::201b
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.24.59|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: 'spa-eng.zip'

spa-eng.zip         100%[===================>]   2.52M  2.44MB/s    in 1.0s    

2024-10-30 13:28:58 (2.44 MB/s) - 'spa-eng.zip' saved [2638744/2638744]



In [3]:
text_file = "spa-eng/spa.txt"

with open(text_file) as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start]" + spanish + "[end]"
    text_pairs.append((english,spanish))

In [4]:
text_pairs[0]

('Go.', '[start]Ve.[end]')

In [5]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15*len(text_pairs))
num_train_samples = len(text_pairs) -2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [6]:
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[","")
strip_chars = strip_chars.replace("]","")

strip_chars

'!"#$%&\'()*+,-./:;<=>?@\\^_`{|}~¿'

In [7]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
max_tokens= vocab_size,
output_mode = "int",
output_sequence_length = sequence_length
)

target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [8]:
train_pairs[:1]

[('Tom left five minutes ago.', '[start]Tom se fue hace cinco minutos.[end]')]

In [9]:
t = [("hello ","hi "),("hello ","hi ")]

for x,y in zip(*train_pairs[:2]):
    print(x,y)

Tom left five minutes ago. Tom lives in a small town not too far from Boston.
[start]Tom se fue hace cinco minutos.[end] [start]Tomás vive en un pueblito no muy lejos de Boston.[end]


In [10]:
batch_size = 64

def format_dataset(eng,spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    
    return({
        "english": eng,
        "spanish": spa[:,:-1],
    },spa[:,1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts,spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

In [11]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [12]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


In [13]:
for inputs, targets in train_ds.take(1):
    print(inputs,targets)

{'english': <tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[ 361,  331,   43, ...,    0,    0,    0],
       [  15,   27,   38, ...,    0,    0,    0],
       [  21,  602,   16, ...,    0,    0,    0],
       ...,
       [ 305,    8,  341, ...,    0,    0,    0],
       [   6,  114,    2, ...,    0,    0,    0],
       [   6,   14, 1157, ...,    0,    0,    0]])>, 'spanish': <tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[  75,   59, 1211, ...,    0,    0,    0],
       [1090,   35, 3529, ...,    0,    0,    0],
       [  27,   19, 2065, ...,    0,    0,    0],
       ...,
       [ 255,    8,  320, ...,    0,    0,    0],
       [   6,   25,   91, ...,    0,    0,    0],
       [   6,   46, 5111, ...,    0,    0,    0]])>} tf.Tensor(
[[  59 1211    1 ...    0    0    0]
 [  35 3529    0 ...    0    0    0]
 [  19 2065    7 ...    0    0    0]
 ...
 [   8  320   20 ...    0    0    0]
 [  25   91    4 ...    0    0    0]
 [  46 5111   18 ...    0    0    0]], shape=(64, 

In [14]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
embed_dim = 256
latent_dim = 1024


class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [15]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [16]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return layers.Lambda(lambda x: tf.not_equal(x, 0))(inputs)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [17]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'transformer_encoder' (of type TransformerEncoder) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [18]:
transformer.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ english             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spanish             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  3,845,120 │ english[0][0]     │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, None)      │          0 │ english[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  3,845,120 │ spanish[0][0]     │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encoder │ (None, None, 256) │  3,155,456 │ positional_embed… │
│ (TransformerEncode… │                   │            │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decoder │ (None, None, 256) │  5,259,520 │ positional_embed… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, None, 256) │          0 │ transformer_deco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, None,      │  3,855,000 │ dropout_3[0][0]   │
│                     │ 15000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 19,960,216 (76.14 MB)

 Trainable params: 19,960,216 (76.14 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
I0000 00:00:1730294967.835701      67 serv

   5/1302 ━━━━━━━━━━━━━━━━━━━━ 41s 32ms/step - accuracy: 0.3978 - loss: 9.2865

I0000 00:00:1730294975.686325      67 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 807/1302 ━━━━━━━━━━━━━━━━━━━━ 14s 29ms/step - accuracy: 0.7562 - loss: 5.8371

W0000 00:00:1730294999.576263      67 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


1301/1302 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7665 - loss: 5.5177

W0000 00:00:1730295020.735178      68 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1730295023.100381      65 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


1302/1302 ━━━━━━━━━━━━━━━━━━━━ 63s 38ms/step - accuracy: 0.7665 - loss: 5.5166 - val_accuracy: 0.8285 - val_loss: 3.5743
Epoch 2/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.8418 - loss: 3.5157 - val_accuracy: 0.8656 - val_loss: 2.8217
Epoch 3/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.8671 - loss: 2.8991 - val_accuracy: 0.8785 - val_loss: 2.5867
Epoch 4/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.8781 - loss: 2.6247 - val_accuracy: 0.8831 - val_loss: 2.5151
Epoch 5/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 40s 31ms/step - accuracy: 0.8851 - loss: 2.4844 - val_accuracy: 0.8829 - val_loss: 2.5107
Epoch 6/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.8885 - loss: 2.4083 - val_accuracy: 0.8857 - val_loss: 2.5253
Epoch 7/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.8923 - loss: 2.3627 - val_accuracy: 0.8867 - val_loss: 2.5456
Epoch 8/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.8954 - loss: 2.32

In [20]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Did you see his face?
[start] su cara[end]                  
-
What sport do you think is the most dangerous?
[start] lo que piensas es el más peligroso que la mayor parte del a la [UNK]   de [UNK] que
-
Bring me today's paper, please.
[start] hoy de hoy[end]                 
-
We are out of danger.
[start]                    
-
I cooked dinner.
[start]                    
-
I went up to the door.
[start] a la puerta del [UNK]               
-
I can buy one.
[start] una mejor[end]                  
-
There was an attempt on the president's life.
[start] un [UNK] sobre el presidente de la vida[end]            
-
He solved the difficult problem easily.
[start] ese problema con facilidad[end]                
-
I like your techno dance moves.
[start] tus [UNK] [UNK] [UNK]                
-
I couldn't refuse.
[start] no podía [UNK]                 
-
Where was this picture taken?
[start] esta foto[end]                  
-
Could I ask you a favor?
[start] usted una lejos                 
-